# Build a nerual net work from scratch using numpy

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 18 09:17:28 2021
@author: Chenfeng Chen

Build a neural network from scratch using numpy.

"""

import numpy as np
import random
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.utils.extmath import softmax
from trading.data.data_folder import data_folder
data_folder = data_folder + "ml_data/"


ModuleNotFoundError: No module named 'trading'

In [3]:

def image_to_1D_array(data):
    """ Convert a torchvision dataset to two numpy arrays x and y, where x is
    the input array and y is the output array. The input array x will be 
    converted from a 2D array to a 1D array and devided by 255 to make all the
    values to be in the range of 0 to 1. 

    Parameters
    ----------
    data : torchvision dataset
        A torchvision non-transformed dataset,
        e.g. torchvision.datasets.MNIST()
        
    Returns
    -------
    x: Input array for a neural network, having a shape[N, H*W], where N is
        the number of the samples in the given training/test data, H is the
        height of a image and W is the width of a image. x is devided by 255
        to squeeze the values to 0 - 1. 
    y: Output array for a neural network.
    """
    n = len(data) # number of samples.
    x_shape = data[0][0].size # shape of x.
    x = np.zeros([n, x_shape[0]*x_shape[1]])
    y = np.zeros(n)
    for i in range(n):
        x[i, :] = np.asarray(data[i][0]).flatten()/255
        y[i, ] = data[i][1]
    return x, y


def number_to_one_hot(data):
    """ Convert number in a torchvision output array to one-hot encoded
    numpy array.    

    Parameters
    ----------
    data : numpy 1D array
        The y output array for a neural network,
        e.g. the y returned from the function image_to_1D_array().

    Returns
    -------
    y : numpy 2D array
        A one-hot encoded array with a shape[N, 10], where N is the number of
        the samples in the given training/test data.
    """
    n = len(data) # number of samples.
    data = data.astype(int) # convert data to int.
    y = np.zeros([n, 10])
    for i in range(n):
        y[i, data[i]] = 1
    return y


def sigmoid(x):
    y = 1/(1 + np.exp(-x))
    return y


def sigmoid_derivative(x):
    y = x*(1-x)    
    return y


def relu(x):
    y = np.copy(x)
    y[x <= 0] = 0
    return y


def relu_derivative(x):
    y = np.ones(x.shape)
    y[x <= 0] = 0
    return y


def softmax_naive(x):
    y = np.exp(x)/np.sum(np.exp(x))
    return y


def softmax_sklearn(X, copy=True):
    """ Calculate the softmax function.
    The softmax function is calculated by
    np.exp(X) / np.sum(np.exp(X), axis=1)
    This will cause overflow when large values are exponentiated.
    Hence the largest value in each row is subtracted from each data
    point to prevent this.
    
    Parameters
    ----------
    X : array-like of float of shape (M, N)
        Argument to the logistic function.
    copy : bool, default=True
        Copy X or not.
    Returns
    -------
    out : ndarray of shape (M, N)
        Softmax function evaluated at every point in x.
    """
    if copy:
        X = np.copy(X)
    max_prob = np.max(X, axis=1).reshape((-1, 1))
    X -= max_prob
    np.exp(X, X)
    sum_prob = np.sum(X, axis=1).reshape((-1, 1))
    X /= sum_prob
    return X


def mse_loss(y, z):
    m = y.shape[0]
    l = np.sum((y - z)**2)/(2*m)
    return l


def cross_entropy_loss(y, z):
    return -np.sum(y*np.log(z))


def calculate_dw(b, a):
    m = b.shape[0]
    n = a.shape[1]
    y = np.zeros([m] + [n, b.shape[1]])
    for i in range(m):
        y[i, ] = b[i, ]*a[i, ].reshape(n, 1)
    return y


def evaluate(x, y):
        # forward(x, y, a=sigmoid):
    h1 = x @ w1 + b1
    h1a = a(h1)
    h2 = h1a @ w2 + b2
    h2a = a(h2)
    o = h2a @ wo + bo
    z = softmax(o) # not using softmax.
    
    # Correct rate.
    cr = np.sum(np.argmax(z, axis=0) == np.argmax(y, axis=0))/y.shape[0]
    print(f"Correct rate: {cr}")
    return cr



In [4]:

# Prepare dataset for neural network traing -----------------------------------

# Import training and test data using torchvision.
train_data = datasets.MNIST(
    root=data_folder,
    train=True,
    download=True,
    transform=None)
test_data = datasets.MNIST(
    root=data_folder,
    train=False,
    download=True,
    transform=None)

# Check the raw data.
x, y = train_data[0]
print(x) # sample x
print(y) # sample y
plt.imshow(x)
x = np.asarray(x)
print(x) # sample x after converting to numpy array.

# tansform torchvision raw data to numpy arrays.
x_train, y_train = image_to_1D_array(data=train_data)
x_test, y_test = image_to_1D_array(data=test_data)

# Convert number to one_hot.
y_train = number_to_one_hot(data=y_train)
y_test = number_to_one_hot(data=y_test)

# Split a validation dataset from the training dataset.
validation_rate = 0.1 # use 10% of training data as validation data.
n_train = int(len(train_data)*(1 - validation_rate))
x_validate = x_train[n_train:]
y_validate = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Processing...


C:\Users\chenf\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:

# Set up neural network parameters --------------------------------------------

# weird: batch size < 4 works better than batch size >= 4， probaly has 
# something to do with updating weights, as the values needs to be the mean of 
# all samples in a batch.
batch_size = 2 

n_x = x_train.shape[1] # number of variables for each input sample.
n_h1 = 64 # number of neurons in the 1st hidden layer.
n_h2 = 32 # number of neurons in the 2nd hidden layer.
n_o = y_train.shape[1] # number of neurons in the output layer.

a = sigmoid # Activation function.
da = sigmoid_derivative # derivative of the activation function.

a = relu # Activation function.
da = relu_derivative # derivative of the activation function.

loss = cross_entropy_loss
#loss = mse_loss

lr = 0.01 # learning rate.

n_samples = range(len(x_train)) # number of samples.



In [ ]:

# Train the neural network ----------------------------------------------------

epoch = 2 # Number of times to train through the full training set.

# make weights small for better training result.
w1 = (np.random.rand(n_x, n_h1) -0.5)/100
b1 = (np.random.rand(n_h1)-0.5)/100
w2 = (np.random.rand(n_h1, n_h2)-0.5)/100
b2 = (np.random.rand(n_h2)-0.5)/100
wo = (np.random.rand(n_h2, n_o)-0.5)/100
bo = (np.random.rand(n_o)-0.5)/100

for i in range(epoch):
    # Get batch_size samples.
    n = len(x_train) # number of samples.
    # shuffle the order of samples randomly.
    random_index = list(range(n))
    random.shuffle(random_index)
    random_index = np.array(random_index)    
    
    batch_list = np.array(range(int(np.floor(n/batch_size))))*batch_size
    for j in batch_list:
        # Slice the batch of x and y for training.
        x = x_train[random_index[j : j + batch_size]]
        y = y_train[random_index[j : j + batch_size]]
        
        # forward(x, y, a=sigmoid):
        h1 = x @ w1 + b1
        h1a = a(h1)
        h2 = h1a @ w2 + b2
        h2a = a(h2)
        o = h2a @ wo + bo
        z = softmax(o)
        l = loss(y, z)
        
        # backpropagation(x, y):        
        # Calculate derivatives.
        dl_dbo = z - y
        dl_db2 = (dl_dbo @ wo.T)*da(h2a)
        dl_db1 = (dl_db2 @ w2.T)*da(h1a)        
        dl_dwo = calculate_dw(b=dl_dbo, a=h2a)
        dl_dw2 = calculate_dw(b=dl_db2, a=h1a)
        dl_dw1 = calculate_dw(b=dl_db1, a=x)
    
        # Update weights and bais.
        bo -= lr*dl_dbo.mean(axis=0)
        wo -= lr*dl_dwo.mean(axis=0)
        b2 -= lr*dl_db2.mean(axis=0)
        w2 -= lr*dl_dw2.mean(axis=0)
        b1 -= lr*dl_db1.mean(axis=0)
        w1 -= lr*dl_dw1.mean(axis=0)
        
        #dl_dbo=dl_db2=dl_db1=dl_dwo=dl_dw2=dl_dw1=0
    print(f"epoch {i+1} loss: {l}")
    
    #evaluate(x=x_validate, y=y_validate)
    x = x_validate  
    y = y_validate          
    h1 = x @ w1 + b1
    h1a = a(h1)
    h2 = h1a @ w2 + b2
    h2a = a(h2)
    o = h2a @ wo + bo
    z = softmax(o)
    
    # Correct rate.
    cr = np.sum(np.argmax(z, axis=1) == np.argmax(y, axis=1))/y.shape[0]
    print(f"Correct rate: {cr}")




frac {partial o} {partial w_o} = h_2a



Forward functions:


$\int_a^b f(x) = F(b) - F(a)$















